In [1]:
from tensorflow import keras
keras.__version__

'2.7.0'

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2117958315770544752
xla_global_id: -1
]


1. 전처리

In [3]:
import os
from glob import glob
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Flatten 
from tensorflow.keras.layers import Conv2D, MaxPooling2D 


In [4]:
base_dir = glob('C:\\workspace\\eatand_medicinal_plant\\학습용데이터\\01.원천데이터\\이미지학습용데이터\\*')
test_dir = glob('C:\\workspace\\eatand_medicinal_plant\\학습용데이터\\01.원천데이터\\이미지학습용데이터\\*')[0]
validation_dir = glob('C:\\workspace\\eatand_medicinal_plant\\학습용데이터\\01.원천데이터\\이미지학습용데이터\\*')[2]
train_dir = glob('C:\\workspace\\eatand_medicinal_plant\\학습용데이터\\01.원천데이터\\이미지학습용데이터\\*')[1]

In [5]:
train_dir

'C:\\workspace\\eatand_medicinal_plant\\학습용데이터\\01.원천데이터\\이미지학습용데이터\\train'

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator # 전처리

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, #  딕셔너리 
    target_size= (200,200), # 타겟 사이즈
    # 한 번에 전처리할 이미지의 수
    batch_size=20,
    # 라벨링 : binary 이진라벨링, categorical 다중라벨링 
    # 라벨링 방법 : 폴더명의 첫 문자의 알파벳으로 0부터 부여
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size= (200,200),
    batch_size=20,
    class_mode='categorical'
)

Found 6070 images belonging to 15 classes.
Found 2037 images belonging to 15 classes.


In [10]:
print(train_generator.class_indices) 
print(test_generator.class_indices)


{'구실잣밤나무': 0, '까마귀쪽나무': 1, '꽝꽝나무': 2, '돈나무': 3, '메밀': 4, '백량금': 5, '순비기나무': 6, '좁은잎천선과': 7, '참가시나무': 8, '참꽃나무': 9, '참식나무': 10, '큰조롱': 11, '한라꽃향유': 12, '해국': 13, '황근': 14}
{'구실잣밤나무': 0, '까마귀쪽나무': 1, '꽝꽝나무': 2, '돈나무': 3, '메밀': 4, '백량금': 5, '순비기나무': 6, '좁은잎천선과': 7, '참가시나무': 8, '참꽃나무': 9, '참식나무': 10, '큰조롱': 11, '한라꽃향유': 12, '해국': 13, '황근': 14}


2. 모델 구축

In [6]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale = 1./255)
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


train_generator = training_datagen.flow_from_directory(
train_dir,
target_size=(150,150),
class_mode='categorical',
batch_size=150
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size= (150,150),
    batch_size=150,
    class_mode='categorical'
)


validation_generator = validation_datagen.flow_from_directory(
validation_dir,
target_size=(150,150),
class_mode='categorical',
batch_size=150
)

Found 6070 images belonging to 15 classes.
Found 2037 images belonging to 15 classes.
Found 2018 images belonging to 15 classes.


In [8]:
model = tf.keras.models.Sequential([
  # Note the input shape is the desired size of the image 150x150 with 3 bytes color
  # This is the first convolution
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  # The second convolution
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  # The third convolution
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  # The fourth convolution
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  # Flatten the results to feed into a DNN
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.5),
  # 512 neuron hidden layer
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(15, activation='softmax')
])


model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator, epochs=100, steps_per_epoch=40, validation_data = validation_generator, verbose = 1, validation_steps=3)

# model.save("image1.h5")

# history = model.fit(train_generator, epochs=50, steps_per_epoch=40, validation_data = validation_generator, verbose = 1, validation_steps=3)
# model.save("image2.h5")


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 74, 74, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 17, 17, 128)     

In [ ]:
history.history

In [9]:
from keras.models import load_model
loadmodel = load_model('image1.h5')

# 3. 모델 사용하기


In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(loadmodel)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)